In [3]:
import os
import json
import requests
import base64
import time
from pathlib import Path
from typing import List, Dict, Any

def find_sqlite_files(base_dir: str) -> List[Dict[str, str]]:
    """
    Tìm tất cả các file SQLite trong thư mục con của thư mục gốc.
    
    Args:
        base_dir: Đường dẫn đến thư mục gốc
        
    Returns:
        Danh sách các từ điển chứa {db_id, file_path}
    """
    sqlite_files = []
    
    # Duyệt qua tất cả các thư mục con
    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        
        # Chỉ xử lý các thư mục
        if os.path.isdir(folder_path):
            # Tìm file .sqlite trong thư mục
            for file_name in os.listdir(folder_path):
                if file_name.endswith('.sqlite'):
                    file_path = os.path.join(folder_path, file_name)
                    sqlite_files.append({
                        'db_id': folder_name,
                        'file_path': file_path
                    })
                    break  # Chỉ lấy file sqlite đầu tiên trong mỗi thư mục
    
    return sqlite_files

def encode_sqlite_file(file_path: str) -> str:
    """
    Đọc file SQLite và mã hóa Base64.
    
    Args:
        file_path: Đường dẫn đến file SQLite
        
    Returns:
        Chuỗi Base64 của file
    """
    with open(file_path, "rb") as file:
        sqlite_binary = file.read()
        return base64.b64encode(sqlite_binary).decode('utf-8')

def process_database(file_info: Dict[str, str], api_url: str, output_dir: str, timeout: int = 3000) -> bool:
    """
    Xử lý một cơ sở dữ liệu và lưu kết quả.
    
    Args:
        file_info: Thông tin về file SQLite
        api_url: URL của API schema-enrichment
        output_dir: Thư mục đầu ra
        timeout: Thời gian chờ tối đa (giây)
        
    Returns:
        True nếu thành công, False nếu thất bại
    """
    db_id = file_info['db_id']
    file_path = file_info['file_path']
    
    print(f"Đang xử lý cơ sở dữ liệu: {db_id}")
    
    try:
        # Mã hóa file SQLite
        sqlite_base64 = encode_sqlite_file(file_path)
        
        # Tạo payload
        payload = {
            "connection_payload": {
                "file": sqlite_base64,
                "dbType": "sqlite"
            }
        }
        
        # Gửi yêu cầu đến API
        response = requests.post(api_url, json=payload, timeout=timeout)
        
        # Kiểm tra xem yêu cầu có thành công không
        if response.status_code == 200:
            result = response.json()
            
            # Tạo tên file đầu ra
            output_file = os.path.join(output_dir, f"{db_id}.json")
            
            # Lưu kết quả vào file JSON
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
                
            print(f"✅ Đã lưu kết quả cho {db_id} vào {output_file}")
            return True
        else:
            print(f"❌ Lỗi khi xử lý {db_id}: {response.status_code} - {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ Lỗi ngoại lệ khi xử lý {db_id}: {str(e)}")
        return False

def main():
    # Cấu hình
    spider_database_path = "E:/Workspace/Repositories/thesis/test/pipeline/SPIDER/database"
    api_url = "http://127.0.0.1:8383/schema-enrichment"
    output_dir = "logs_schema_enrich"
    
    # Tạo thư mục đầu ra nếu chưa tồn tại
    os.makedirs(output_dir, exist_ok=True)
    
    # Tìm tất cả file SQLite
    sqlite_files = find_sqlite_files(spider_database_path)
    print(f"Tìm thấy {len(sqlite_files)} cơ sở dữ liệu SQLite")
    
    # Thống kê
    total = len(sqlite_files)
    success = 0
    failed = 0
    
    # Xử lý từng cơ sở dữ liệu
    for i, file_info in enumerate(sqlite_files):
        print(f"\nXử lý cơ sở dữ liệu {i+1}/{total}: {file_info['db_id']}")
        
        # Xử lý cơ sở dữ liệu
        result = process_database(file_info, api_url, output_dir)
        
        if result:
            success += 1
        else:
            failed += 1
        
        # Tạm nghỉ giữa các yêu cầu để tránh quá tải
        if i < total - 1:
            time.sleep(2)
    
    # In thống kê
    print(f"\n===== Kết quả =====")
    print(f"Tổng số: {total}")
    print(f"Thành công: {success}")
    print(f"Thất bại: {failed}")

if __name__ == "__main__":
    main()

Tìm thấy 166 cơ sở dữ liệu SQLite

Xử lý cơ sở dữ liệu 1/166: academic
Đang xử lý cơ sở dữ liệu: academic
✅ Đã lưu kết quả cho academic vào logs_schema_enrich\academic.json

Xử lý cơ sở dữ liệu 2/166: activity_1
Đang xử lý cơ sở dữ liệu: activity_1
✅ Đã lưu kết quả cho activity_1 vào logs_schema_enrich\activity_1.json

Xử lý cơ sở dữ liệu 3/166: aircraft
Đang xử lý cơ sở dữ liệu: aircraft
✅ Đã lưu kết quả cho aircraft vào logs_schema_enrich\aircraft.json

Xử lý cơ sở dữ liệu 4/166: allergy_1
Đang xử lý cơ sở dữ liệu: allergy_1
✅ Đã lưu kết quả cho allergy_1 vào logs_schema_enrich\allergy_1.json

Xử lý cơ sở dữ liệu 5/166: apartment_rentals
Đang xử lý cơ sở dữ liệu: apartment_rentals
✅ Đã lưu kết quả cho apartment_rentals vào logs_schema_enrich\apartment_rentals.json

Xử lý cơ sở dữ liệu 6/166: architecture
Đang xử lý cơ sở dữ liệu: architecture
✅ Đã lưu kết quả cho architecture vào logs_schema_enrich\architecture.json

Xử lý cơ sở dữ liệu 7/166: assets_maintenance
Đang xử lý cơ sở dữ l

In [9]:
# json dump result
import json
with open('schema_enrichment_results.json', 'w') as f:
    json.dump(response.json(), f, indent=2)

In [1]:
import os
import requests
import sys
import time
import json
import networkx as nx
import community as community_louvain # Thư viện python-louvain
import matplotlib.pyplot as plt
import random
import traceback

# --- Cấu hình ---
# !!! QUAN TRỌNG: Thay đổi đường dẫn này đến thư mục 'database' của bộ dữ liệu Spider trên máy bạn !!!
spider_database_path = "E:/Workspace/Repositories/thesis/test/pipeline/SPIDER/database" 
# Ví dụ: spider_database_path = "C:/Users/YourUser/Downloads/spider/database" 
#    hoặc spider_database_path = "/home/youruser/datasets/spider/database"

api_url = "http://localhost:8181/api/v1/db/get-schema/sqlite"
# --- Kết thúc cấu hình ---

# --- Hàm gọi API để lấy Schema ---
def fetch_schema_from_api(db_file_path, endpoint_url):
    """
    Đọc file SQLite, gửi đến API và trả về phần 'data' của schema nếu thành công.
    """
    try:
        with open(db_file_path, 'rb') as f:
            db_content = f.read()
        
        if not db_content:
             print(f"  Lỗi: File rỗng hoặc không đọc được: {db_file_path}")
             return None

        file_name = os.path.basename(db_file_path)
        files = {'file': (file_name, db_content, 'application/octet-stream')}

        print(f"  Đang gửi '{file_name}' đến API...")
        response = requests.post(endpoint_url, files=files, timeout=60) 
        response.raise_for_status() 
        result = response.json()

        if result.get("code") == 0:
            schema_data = result.get("data", {})
            if not schema_data or 'tables' not in schema_data: # Kiểm tra có 'tables' không
                print(f"  Lỗi: API trả về mã thành công nhưng thiếu dữ liệu 'tables' cho '{file_name}'.")
                return None
            print(f"  Thành công: Đã lấy schema cho '{file_name}'.")
            return schema_data # Chỉ trả về phần 'data' chứa schema
        else:
            error_message = result.get("message", "Lỗi không xác định từ API")
            print(f"  Lỗi API: '{file_name}' -> {error_message} (Mã lỗi: {result.get('code')})")
            return None

    except FileNotFoundError:
        print(f"  Lỗi: Không tìm thấy file: {db_file_path}")
        return None
    except requests.exceptions.ConnectionError:
        print(f"  Lỗi: Không thể kết nối đến API tại {endpoint_url}.")
        return None # Có thể thêm sys.exit(1) nếu muốn dừng hẳn
    except requests.exceptions.Timeout:
        print(f"  Lỗi: Yêu cầu đến API bị timeout cho file '{file_name}'.")
        return None
    except requests.exceptions.RequestException as e:
        print(f"  Lỗi Request: '{file_name}' -> {e}")
        return None
    except requests.exceptions.JSONDecodeError:
        print(f"  Lỗi: Không thể giải mã JSON từ phản hồi API cho '{file_name}'.")
        print(f"  Phản hồi nhận được (100 ký tự đầu): {response.text[:100]}...")
        return None
    except Exception as e:
        print(f"  Lỗi không mong muốn khi gọi API: '{file_name}' -> {e}")
        return None

## --- Hàm thực hiện phân cụm từ dữ liệu Schema (ĐÃ CẬP NHẬT)---
def cluster_schema_from_data(schema_data, 
                             db_identifier="Unknown DB", 
                             plot_graph=False, 
                             output_dir="cluster_plots", 
                             resolution_value=1.0):
    """
    Phân tích schema_data, xây dựng đồ thị và phân cụm với resolution tùy chỉnh.
    Input: schema_data (dict), db_identifier (str), plot_graph (bool), resolution_value (float)
    Output: Danh sách các cụm (list of lists) hoặc None nếu lỗi.
    """
    print(f"  -> Đang phân cụm schema cho '{db_identifier}'...")
    try:
        tables_info = schema_data.get('tables', [])
        if not tables_info:
             print("    Lỗi: Không tìm thấy thông tin 'tables' trong schema_data.")
             return [] 
        
        tables = [table['tableIdentifier'] for table in tables_info]
        
        foreign_keys = []
        for table_info in tables_info:
            source_table = table_info['tableIdentifier']
            for column_info in table_info.get('columns', []):
                if column_info.get('relations'): 
                    for relation in column_info['relations']:
                        target_table = relation.get('tableIdentifier')
                        if target_table and source_table in tables and target_table in tables and source_table != target_table:
                            foreign_keys.append((source_table, target_table))
        
        foreign_keys = list(set(foreign_keys)) 
        print(f"    Trích xuất: {len(tables)} bảng, {len(foreign_keys)} khóa ngoại.")

        # --- Bước 2: Xây dựng Đồ thị ---
        G = nx.Graph()
        G.add_nodes_from(tables)
        G.add_edges_from(foreign_keys)

        if G.number_of_nodes() == 0:
            print("    Thông báo: Không có bảng nào để tạo đồ thị.")
            return []

        # --- Bước 3: Áp dụng Thuật toán Cộng đồng (Louvain) ---
        if G.number_of_edges() > 0:
            print(f"    Áp dụng Louvain với resolution = {resolution_value}") 
            partition = community_louvain.best_partition(G, resolution=resolution_value)
        else:
            partition = {node: i for i, node in enumerate(G.nodes())}
            print("    Cảnh báo: Đồ thị không có cạnh. Mỗi bảng là một cụm riêng.")
            
        # --- Bước 4: Xử lý Kết quả Phân cụm ---
        communities = {}
        for node, community_id in partition.items():
            if community_id not in communities:
                communities[community_id] = []
            communities[community_id].append(node)

        print(f"    Phát hiện được {len(communities)} cụm.")
        cluster_list = []
        sorted_communities = sorted(communities.values(), key=len, reverse=True)

        for i, nodes in enumerate(sorted_communities):
            sorted_nodes = sorted(nodes) 
            print(f"      Cụm {i + 1} (Size: {len(sorted_nodes)}): {sorted_nodes}") 
            cluster_list.append(sorted_nodes)
            
        # --- Bước 5: Vẽ đồ thị (Tùy chọn) ---
        if plot_graph and G.number_of_nodes() > 0:
            try:
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                    print(f"    Đã tạo thư mục lưu đồ thị: {output_dir}")

                # Tăng kích thước hình để có nhiều không gian hơn
                plt.figure(figsize=(20, 16))
                
                # Sử dụng spring_layout với k nhỏ hơn để nén các cạnh lại
                # k là tham số quyết định khoảng cách giữa các nút, giá trị nhỏ hơn sẽ tạo đồ thị gọn hơn
                pos = nx.spring_layout(G, k=0.3, seed=42)  # seed cho kết quả nhất quán
                # Hoặc có thể dùng nx.kamada_kawai_layout nhưng với điều chỉnh tỷ lệ:
                # pos = nx.kamada_kawai_layout(G)
                # pos = {node: (x*0.8, y*0.8) for node, (x, y) in pos.items()}  # Thu nhỏ khoảng cách

                unique_community_ids = set(partition.values())
                num_communities = len(unique_community_ids)
                colors = plt.cm.get_cmap('tab20', max(num_communities, 2))  # Sử dụng bảng màu phong phú hơn
                
                community_color_map = {community_id: colors(i % 20) for i, community_id in enumerate(unique_community_ids)}
                node_colors = [community_color_map[partition[node]] for node in G.nodes()]

                # Vẽ node với kích thước và phông chữ lớn hơn
                nx.draw_networkx_nodes(G, pos, 
                                     node_size=2000,  # Tăng kích thước node
                                     node_color=node_colors,
                                     alpha=0.8)  # Độ trong suốt nhẹ
                
                # Vẽ các cạnh ngắn hơn với màu nhạt hơn
                nx.draw_networkx_edges(G, pos,
                                     width=1.5,  # Tăng độ dày của cạnh
                                     alpha=0.5,  # Giảm độ đậm để không lấn át node
                                     edge_color='grey')
                
                # Vẽ nhãn với font chữ lớn hơn và đậm hơn
                nx.draw_networkx_labels(G, pos, 
                                      font_size=14,  # Tăng kích thước font
                                      font_weight='bold',
                                      font_family='sans-serif')
                
                # Điều chỉnh giới hạn trục để đảm bảo nhãn không bị cắt
                plt.axis('off')  # Tắt trục
                plt.tight_layout()  # Tối ưu việc sử dụng không gian
                
                # Tạo tên file an toàn từ db_identifier
                safe_db_name = "".join(c if c.isalnum() else "_" for c in db_identifier)
                plot_filename = os.path.join(output_dir, f"clusters_{safe_db_name}.png")
                plt.title(f"Clusters for {db_identifier}", fontsize=18, fontweight='bold')  # Tăng kích thước tiêu đề
                
                # Lưu hình với DPI cao hơn
                plt.savefig(plot_filename, dpi=150, bbox_inches='tight')
                plt.close()
                print(f"    Đã lưu đồ thị phân cụm vào: {plot_filename}")
            except Exception as plot_err:
                print(f"    Lỗi khi vẽ hoặc lưu đồ thị: {plot_err}")

        return cluster_list

    except Exception as e:
        print(f"  Lỗi trong quá trình phân cụm cho '{db_identifier}': {e}")
        traceback.print_exc()
        return None

# --- Logic chính của Script ---
if not os.path.isdir(spider_database_path):
    print(f"LỖI: Đường dẫn '{spider_database_path}' không tồn tại hoặc không phải là thư mục.")
    sys.exit(1)

print(f"Bắt đầu quét và lấy schema từ các file SQLite trong: {spider_database_path}")
print(f"API Endpoint: {api_url}")

processed_db_count = 0
fetch_success_count = 0
cluster_success_count = 0
failed_items = [] # Lưu trữ tên file và lý do thất bại
all_database_clusters = {} # Lưu kết quả cluster: {db_full_path: cluster_list}

# --- Cờ để bật/tắt vẽ đồ thị ---
GENERATE_PLOTS = True # Đặt thành True nếu bạn muốn tạo file ảnh đồ thị cho mỗi DB
PLOT_OUTPUT_DIR = "spider_cluster_plots" # Thư mục lưu ảnh đồ thị (nếu GENERATE_PLOTS=True)
# -----------------------------

for dirpath, dirnames, filenames in os.walk(spider_database_path):
    filenames.sort() 
    for filename in filenames:
        if filename.lower().endswith(".sqlite"):
            db_full_path = os.path.join(dirpath, filename)
            db_folder_name = os.path.basename(dirpath)
            db_identifier = f"{db_folder_name}/{filename}" # Định danh cho database
            
            print(f"\nProcessing: {db_identifier}")
            processed_db_count += 1
            
            # 1. Lấy schema từ API
            schema_data = fetch_schema_from_api(db_full_path, api_url)
            
            if schema_data:
                fetch_success_count += 1
                # 2. Nếu lấy schema thành công, tiến hành phân cụm
                clusters = cluster_schema_from_data(schema_data, 
                                                    db_identifier=db_identifier, 
                                                    plot_graph=GENERATE_PLOTS,
                                                    output_dir=PLOT_OUTPUT_DIR)
                
                if clusters is not None:
                    # Phân cụm thành công
                    cluster_success_count += 1
                    all_database_clusters[db_full_path] = clusters
                else:
                    # Phân cụm thất bại (dù đã lấy được schema)
                    failed_items.append(f"{db_full_path} (Lỗi phân cụm)")
            else:
                # Lấy schema thất bại
                failed_items.append(f"{db_full_path} (Lỗi API/Fetch)")

            # time.sleep(0.05) # Thêm delay nhỏ nếu cần

print("\n--- Hoàn tất xử lý ---")
print(f"Tổng số file .sqlite đã xử lý: {processed_db_count}")
print(f"Số lượng lấy schema thành công: {fetch_success_count}")
print(f"Số lượng phân cụm thành công: {cluster_success_count}")
print(f"Số lượng xử lý thất bại (API hoặc phân cụm): {len(failed_items)}")

if failed_items:
    print("\nDanh sách các mục xử lý thất bại:")
    for item in failed_items:
        print(f"- {item}")

# Bây giờ biến `all_database_clusters` chứa kết quả phân cụm cho mỗi database
# Ví dụ cách truy cập kết quả:
# if all_database_clusters:
#     first_db = list(all_database_clusters.keys())[0]
#     print(f"\nVí dụ kết quả phân cụm cho database đầu tiên ({first_db}):")
#     for i, cluster in enumerate(all_database_clusters[first_db]):
#         print(f"  Cluster {i+1}: {cluster}")

Bắt đầu quét và lấy schema từ các file SQLite trong: E:/Workspace/Repositories/thesis/test/pipeline/SPIDER/database
API Endpoint: http://localhost:8181/api/v1/db/get-schema/sqlite

Processing: academic/academic.sqlite
  Đang gửi 'academic.sqlite' đến API...
  Thành công: Đã lấy schema cho 'academic.sqlite'.
  -> Đang phân cụm schema cho 'academic/academic.sqlite'...
    Trích xuất: 15 bảng, 17 khóa ngoại.
    Áp dụng Louvain với resolution = 1.0
    Phát hiện được 6 cụm.
      Cụm 1 (Size: 3): ['author', 'domain_author', 'writes']
      Cụm 2 (Size: 3): ['conference', 'domain', 'domain_conference']
      Cụm 3 (Size: 3): ['domain_keyword', 'keyword', 'publication_keyword']
      Cụm 4 (Size: 3): ['cite', 'domain_publication', 'publication']
      Cụm 5 (Size: 2): ['domain_journal', 'journal']
      Cụm 6 (Size: 1): ['organization']
    Đã tạo thư mục lưu đồ thị: spider_cluster_plots


C:\Users\gkiet\AppData\Local\Temp\ipykernel_2280\1599822436.py:159: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab20', max(num_communities, 2))  # Sử dụng bảng màu phong phú hơn


    Đã lưu đồ thị phân cụm vào: spider_cluster_plots\clusters_academic_academic_sqlite.png

Processing: activity_1/activity_1.sqlite
  Đang gửi 'activity_1.sqlite' đến API...
  Thành công: Đã lấy schema cho 'activity_1.sqlite'.
  -> Đang phân cụm schema cho 'activity_1/activity_1.sqlite'...
    Trích xuất: 5 bảng, 4 khóa ngoại.
    Áp dụng Louvain với resolution = 1.0
    Phát hiện được 2 cụm.
      Cụm 1 (Size: 3): ['Activity', 'Participates_in', 'Student']
      Cụm 2 (Size: 2): ['Faculty', 'Faculty_Participates_in']
    Đã lưu đồ thị phân cụm vào: spider_cluster_plots\clusters_activity_1_activity_1_sqlite.png

Processing: aircraft/aircraft.sqlite
  Đang gửi 'aircraft.sqlite' đến API...
  Thành công: Đã lấy schema cho 'aircraft.sqlite'.
  -> Đang phân cụm schema cho 'aircraft/aircraft.sqlite'...
    Trích xuất: 5 bảng, 4 khóa ngoại.
    Áp dụng Louvain với resolution = 1.0
    Phát hiện được 2 cụm.
      Cụm 1 (Size: 3): ['aircraft', 'airport', 'airport_aircraft']
      Cụm 2 (Size: 

In [1]:
from llama_index.utils.workflow import draw_all_possible_flows
from core.workflow import SQLAgentWorkflow, SchemaEnrichmentWorkflow

draw_all_possible_flows(SQLAgentWorkflow, filename="SQLAgentWorkflow.html")
draw_all_possible_flows(SchemaEnrichmentWorkflow, filename="SchemaEnrichmentWorkflow.html")

<class 'core.workflow.SQLReflectionEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class 'core.workflow.SQLValidatorEvent'>
<class 'core.workflow.SQLValidatorEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class 'core.workflow.TextToSQLEvent'>
<class 'core.workflow.TableRetrieveEvent'>
<class 'core.workflow.TextToSQLEvent'>
<class 'core.workflow.ExecuteSQLEvent'>
<class 'NoneType'>
SQLAgentWorkflow.html
<class 'llama_index.core.workflow.events.StopEvent'>
<class 'core.workflow.SchemaEnrichmentEvent'>
<class 'NoneType'>
SchemaEnrichmentWorkflow.html


In [2]:
import json
import base64
from typing import List, Dict, Any

# Import từ service.py của bạn
from core.services import get_schema, execute_sql, validate_connection_payload
from core.templates import SCHEMA_ENRICHMENT_TMPL
from core.utils import show_prompt

def get_table_examples(connection_payload, table_name, columns, limit=3, max_value_length=50):
    """Lấy các giá trị ví dụ từ bảng được chỉ định theo thứ tự cột cụ thể"""
    try:
        # Lấy tên cột từ danh sách columns (bỏ kiểu dữ liệu và các thông tin khác)
        column_names = []
        for col_def in columns:
            # Tách tên cột từ chuỗi định nghĩa (ví dụ: "id INTEGER PRIMARY KEY" -> "id")
            col_name = col_def.split()[0]
            column_names.append(col_name)
        
        # Tạo câu truy vấn SELECT với các cột cụ thể
        columns_query = ", ".join(column_names)
        query = f"SELECT {columns_query} FROM {table_name} LIMIT {limit}"
        
        result = execute_sql(connection_payload, query)
        
        # Kiểm tra lỗi
        if result.get("error"):
            return []
        
        # Truy cập trực tiếp đến data
        data = result.get("data", [])
        
        # Định dạng dữ liệu theo kiểu CSV với thứ tự cột giống như trong cấu trúc bảng
        csv_rows = []
        for record in data:
            # Tạo một hàng với các giá trị theo thứ tự cột
            row_values = []
            for col_name in column_names:
                value = record.get(col_name, "")
                # Chuyển đổi giá trị thành chuỗi
                str_value = str(value) if value is not None else "NULL"
                
                # Cắt ngắn chuỗi nếu quá dài
                if len(str_value) > max_value_length:
                    str_value = str_value[:max_value_length] + "..."
                
                # Thêm dấu ngoặc kép nếu giá trị chứa dấu phẩy
                if "," in str_value:
                    str_value = f'"{str_value}"'
                    
                row_values.append(str_value)
            
            # Nối các giá trị bằng dấu phẩy để tạo hàng CSV
            csv_rows.append(", ".join(row_values))
        
        return csv_rows
    except Exception as e:
        print(f"Error getting examples for table {table_name}: {str(e)}")
        return []

def create_table_relationship_arrays(table_details):
    """Tạo các mảng quan hệ của bảng với bảng chính ở vị trí đầu tiên"""
    # Khởi tạo dictionary để lưu mối quan hệ của từng bảng
    table_relationships = {}
    
    # Duyệt qua mỗi bảng trong schema
    for table in table_details:
        table_name = table["tableIdentifier"]
        # Khởi tạo mảng rỗng cho mỗi bảng
        table_relationships[table_name] = []
        
    # Duyệt lại từng bảng để tìm các mối quan hệ
    for table in table_details:
        table_name = table["tableIdentifier"]
        
        # Duyệt qua từng cột trong bảng
        for column in table["columns"]:
            if "relations" in column and column["relations"]:
                # Nếu cột có quan hệ, thêm bảng liên kết vào mảng
                for relation in column["relations"]:
                    related_table = relation["tableIdentifier"]
                    
                    # Thêm mối quan hệ hai chiều
                    if related_table not in table_relationships[table_name]:
                        table_relationships[table_name].append(related_table)
                    if table_name not in table_relationships[related_table]:
                        table_relationships[related_table].append(table_name)
    
    # Chuyển đổi từ dictionary thành mảng 2 chiều
    relationship_arrays = []
    for table_name, related_tables in table_relationships.items():
        # Tạo một mảng con với bảng chính ở vị trí đầu tiên
        array = [table_name] + related_tables
        relationship_arrays.append(array)
    
    # Đếm số lần xuất hiện của mỗi bảng trong tất cả các mảng quan hệ
    table_occurrence_count = {}
    for arr in relationship_arrays:
        for table in arr:
            if table in table_occurrence_count:
                table_occurrence_count[table] += 1
            else:
                table_occurrence_count[table] = 1
    
    # Tạo mảng chứa các bảng xuất hiện từ 2 lần trở lên
    duplicate_tables = [table for table, count in table_occurrence_count.items() if count >= 2]
    
    # Thêm mảng các bảng xuất hiện nhiều lần vào kết quả
    result = {
        "relationship_arrays": relationship_arrays,
        "duplicate_tables": duplicate_tables
    }
    
    return result

def create_schema_enrichment_prompts(table_details, table_groups, connection_payload):
    """Tạo các prompt cho việc phân tích ngữ nghĩa của schema với example values"""
    prompts = []
    
    for group in table_groups:
        main_table = group[0]
        related_tables = group[1:]
        
        # Get table structures and examples
        table_structures = {}
        table_examples = {}
        
        for table in table_details:
            if table["tableIdentifier"] in group:
                # Get structure
                columns = []
                for column in table["columns"]:
                    column_type = column["columnType"]
                    pk_indicator = " PRIMARY KEY" if column["isPrimaryKey"] else ""
                    columns.append(f"{column['columnIdentifier']} {column_type}{pk_indicator}")
                table_structures[table["tableIdentifier"]] = columns
                
                # Get examples with the same column order
                examples = get_table_examples(connection_payload, table["tableIdentifier"], columns)
                table_examples[table["tableIdentifier"]] = examples
        
        # Format the table group information
        group_info = f"Table group: {group}\n"
        for table_name in group:
            if table_name in table_structures:
                columns_str = ", ".join(table_structures[table_name])
                group_info += f"- {table_name}: ({columns_str})\n"
                
            # Add example values if available
            if table_examples.get(table_name):
                group_info += f"\n* Example Values for {table_name} (CSV format):\n"
                # First add column headers
                column_headers = ", ".join([col.split()[0] for col in table_structures[table_name]])
                group_info += f"{column_headers}\n"
                
                # Then add data rows
                for example in table_examples[table_name]:
                    group_info += f"{example}\n"
                group_info += "\n"
        
        # Create the prompt with simplified structured response format
        prompt = SCHEMA_ENRICHMENT_TMPL.format_messages(
            group_info=group_info
        )
    
        prompts.append({
            "main_table": main_table,
            "related_tables": related_tables,
            "prompt": prompt
        })
    
    return prompts

def parse_llm_json_response(response_text):
    try:
        # Tìm JSON trong phản hồi
        json_start = response_text.find('```json')
        if json_start != -1:
            # Nếu JSON được bọc trong dấu ```json ... ```
            json_content = response_text[json_start + 7:]  # +7 để bỏ qua ```json
            json_end = json_content.find('```')
            if json_end != -1:
                json_content = json_content[:json_end].strip()
            return json.loads(json_content)
        else:
            # Thử tìm '{'
            json_start = response_text.find('{')
            if json_start != -1:
                json_end = response_text.rfind('}') + 1
                if json_end > json_start:
                    json_content = response_text[json_start:json_end].strip()
                    return json.loads(json_content)
        
        # Thử phân tích toàn bộ chuỗi phản hồi
        return json.loads(response_text)
    
    except json.JSONDecodeError as e:
        print(f"Lỗi phân tích JSON: {e}")
        return None
    except Exception as e:
        print(f"Lỗi không mong đợi: {e}")
        return None

# Kết nối SQLite database
sqlite_file_path = r"E:\Workspace\Repositories\thesis\test\pipeline\SPIDER\database\bike_1\bike_1.sqlite"

connection_payload = {
    "dbType": "sqlite",
    "file": None  # Sẽ được điền sau khi đọc file
}

# Đọc SQLite file và encode base64
try:
    with open(sqlite_file_path, "rb") as file:
        file_content = file.read()
        connection_payload["file"] = base64.b64encode(file_content).decode('utf-8')
except Exception as e:
    print(f"Error reading SQLite file: {str(e)}")
    print("Please provide a valid SQLite file path.")

# Kiểm tra connection payload
is_valid, error_msg = validate_connection_payload(connection_payload)
if not is_valid:
    print(f"Invalid connection payload: {error_msg}")
else:
    # Lấy schema của database
    table_details = get_schema(connection_payload)
    
    # Tạo các mảng quan hệ bảng
    table_groups = create_table_relationship_arrays(table_details)["relationship_arrays"]
    
    # Tạo các prompt phân tích ngữ nghĩa với example values
    prompts = create_schema_enrichment_prompts(table_details, table_groups, connection_payload)
    
    
    # In ví dụ prompt
    if prompts:
        print("Example prompt for the first table group:")
        print(prompts[0]["prompt"])
        print("\nTotal prompts generated:", len(prompts))
        print("All prompts saved to 'schema_enrichment_prompts.json'")

2025-04-23 14:52:43,970 - core.services - INFO - EMBED_HOST_API: http://localhost:8181


Example prompt for the first table group:
[ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='# Task: Analyze database schema semantics\n\n## Schema Information:\nDatabase context: {db_info}\n\nTables and columns:\n{cluster_info}\n\n## Instructions:\n1. Analyze each table and column\'s semantic purpose\n2. Use brief and clear semantic descriptions for each table and column\n3. Format as JSON exactly as shown below\n\n## Response Format:\n```json\n[\n  {{"table_name": "TABLE1", "description": "brief table purpose", "columns": [\n    {{"column_name": "COLUMN1", "description": "brief column purpose"}},\n    {{"column_name": "COLUMN2", "description": "brief column purpose"}}\n  ]}},\n  {{"table_name": "TABLE2", "description": "brief table purpose", "columns": [\n    {{"column_name": "COLUMN1", "description": "brief column purpose"}}\n  ]}}\n]\n```\n\nReturn only the JSON output with no additional text.')])]

Total prompts generated

In [11]:
print(create_table_relationship_arrays(table_details)["duplicate_tables"])

['station', 'status']


In [12]:

for prompt in prompts:
    result = []
    show_prompt(prompt["prompt"])
    chat_response = llm.chat(prompt["prompt"])
    semantic = parse_llm_json_response(chat_response.message.content)
    print(semantic)




Formatted Prompt Messages:
--------------------------------------------------------------------------------
Role: user
Content:
Analyze these database tables and their columns to determine their semantic meaning: 

Table group: ['station', 'status']
- station: (id INTEGER PRIMARY KEY, name TEXT, lat NUMERIC, long NUMERIC, dock_count INTEGER, city TEXT, installation_date TEXT)

* Example Values for station (CSV format):
id, name, lat, long, dock_count, city, installation_date
2, San Jose Diridon Caltrain Station, 37.329732, -121.90178200000001, 27, San Jose, 8/6/2013
3, San Jose Civic Center, 37.330698, -121.888979, 15, San Jose, 8/5/2013
4, Santa Clara at Almaden, 37.333988, -121.894902, 11, San Jose, 8/6/2013

- status: (station_id INTEGER, bikes_available INTEGER, docks_available INTEGER, time TEXT)

* Example Values for status (CSV format):
station_id, bikes_available, docks_available, time
3, 12, 3, 2015-06-02 12:46:02
3, 12, 3, 2015-06-02 12:47:02
3, 12, 3, 2015-06-02 12:48:02




2025-04-16 16:09:13,907 - httpx - INFO - HTTP Request: POST https://prepared-anemone-routinely.ngrok-free.app/api/chat "HTTP/1.1 200 OK"


{'tables': {'station': {'description': 'Contains information about bike stations including their location, capacity, and city.', 'columns': {'id': 'Unique identifier for each station.', 'name': 'Name of the station.', 'lat': "Latitude coordinate of the station's location.", 'long': "Longitude coordinate of the station's location.", 'dock_count': 'Total number of docks available at the station.', 'city': 'City where the station is located.', 'installation_date': 'Date when the station was installed.'}}, 'status': {'description': 'Records the current status of each bike station including available bikes and docks, and the time of recording.', 'columns': {'station_id': "Foreign key linking to the 'id' in the 'station' table, identifying which station this record corresponds to.", 'bikes_available': 'Number of bikes currently available at the station.', 'docks_available': 'Number of docks currently available for use at the station.', 'time': 'Timestamp when the status was recorded.'}}}}

F

2025-04-16 16:09:23,926 - httpx - INFO - HTTP Request: POST https://prepared-anemone-routinely.ngrok-free.app/api/chat "HTTP/1.1 200 OK"


{'tables': {'status': {'description': 'Tracks the current status of bike stations, including the number of bikes available and docks free, as well as the timestamp.', 'columns': {'station_id': 'Unique identifier for a specific bike station.', 'bikes_available': 'Number of bicycles currently available at the station.', 'docks_available': 'Number of bike docks that are empty at the station.', 'time': 'Timestamp when the status data was recorded.'}}, 'station': {'description': 'Provides details about individual bike stations, including their location and capacity.', 'columns': {'id': 'Unique identifier for a specific bike station.', 'name': 'Name of the bike station.', 'lat': "Latitude coordinate of the station's location.", 'long': "Longitude coordinate of the station's location.", 'dock_count': 'Total number of docks available at the station.', 'city': 'City in which the station is located.', 'installation_date': 'Date when the bike station was installed.'}}}}

Formatted Prompt Messages

2025-04-16 16:09:32,220 - httpx - INFO - HTTP Request: POST https://prepared-anemone-routinely.ngrok-free.app/api/chat "HTTP/1.1 200 OK"


{'tables': {'trip': {'description': 'Stores information about bike trips.', 'columns': {'id': 'Unique identifier for each trip.', 'duration': 'Duration of the trip in seconds.', 'start_date': 'Date and time when the trip started.', 'start_station_name': 'Name of the station where the trip began.', 'start_station_id': 'Identifier for the starting station.', 'end_date': 'Date and time when the trip ended.', 'end_station_name': 'Name of the station where the trip ended.', 'end_station_id': 'Identifier for the ending station.', 'bike_id': 'Identifier for the bike used in the trip.', 'subscription_type': 'Type of subscription (e.g., Subscriber, Customer).', 'zip_code': 'ZIP code associated with the starting point of the trip.'}}}}

Formatted Prompt Messages:
--------------------------------------------------------------------------------
Role: user
Content:
Analyze these database tables and their columns to determine their semantic meaning: 

Table group: ['weather']
- weather: (date TEXT, 

2025-04-16 16:09:49,048 - httpx - INFO - HTTP Request: POST https://prepared-anemone-routinely.ngrok-free.app/api/chat "HTTP/1.1 200 OK"


{'tables': {'weather': {'description': 'A table containing daily weather records.', 'columns': {'date': 'The date of the weather observation.', 'max_temperature_f': 'The maximum temperature in Fahrenheit.', 'mean_temperature_f': 'The average temperature in Fahrenheit.', 'min_temperature_f': 'The minimum temperature in Fahrenheit.', 'max_dew_point_f': 'The maximum dew point in Fahrenheit.', 'mean_dew_point_f': 'The average dew point in Fahrenheit.', 'min_dew_point_f': 'The minimum dew point in Fahrenheit.', 'max_humidity': 'The maximum humidity percentage.', 'mean_humidity': 'The average humidity percentage.', 'min_humidity': 'The minimum humidity percentage.', 'max_sea_level_pressure_inches': 'The maximum sea level pressure in inches.', 'mean_sea_level_pressure_inches': 'The average sea level pressure in inches.', 'min_sea_level_pressure_inches': 'The minimum sea level pressure in inches.', 'max_visibility_miles': 'The maximum visibility distance in miles.', 'mean_visibility_miles': 'T

In [8]:
semantic = parse_llm_json_response(chat_response.message.content)
print(semantic)

{'tables': {'status': {'description': 'Tracks the current status of each station in terms of available bikes and docks at specific times.', 'columns': {'station_id': 'Unique identifier for a station.', 'bikes_available': 'Number of bikes currently available at the station.', 'docks_available': 'Number of docks currently available at the station.', 'time': 'Timestamp when the status was recorded.'}}, 'station': {'description': 'Provides detailed information about each bike sharing station, including its location and capacity.', 'columns': {'id': 'Unique identifier for a station.', 'name': 'Name of the station.', 'lat': "Latitude coordinate of the station's location.", 'long': "Longitude coordinate of the station's location.", 'dock_count': 'Total number of docks at the station.', 'city': 'City in which the station is located.', 'installation_date': 'Date when the station was installed.'}}}}
